# LAB GenAI - LLMs - OpenAI Assistant

## Exercise 1

Create an assistant to answer a topic of your choosing:
 - Upload a file of your interest
 - Add Instructions to the prompt
 - Use the assistant in Playground mode

 https://platform.openai.com/playground/assistants

In [20]:
import openai
import pandas as pd

#OpenAI API key
api_key = ''

In [23]:
## Create the assistant
from openai import OpenAI
 
client = OpenAI(api_key=api_key)
 
assistant = client.beta.assistants.create(
  name="Pickleball Coaching Assistant",
  instructions="You are an expert in pickleball coaching and certification. Answer questions using the uploaded PPR Study Guide.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [37]:
# Create a vector store called "Datavisualization Documents"
vector_store = client.beta.vector_stores.create(name="pickl")

In [32]:
# Ready the files for upload to OpenAI
file_paths = ["/Users/paolarivera/Documents/Ironhack/Week 8/Day 2/lab-genai-llms-openai-assistant-main/your-code/PPR_StudyGuide_V10.pdf"]
file_streams = [open(path, "rb") for path in file_paths]


In [39]:
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [41]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

## Exercise 2

Talk to your assistant via the API

https://platform.openai.com/docs/assistants/overview

In [44]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What are the key skills required to become a certified pickleball coach?",
    }
  ]
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
print(message_content.value)

IndexError: list index out of range

In [46]:
from openai import OpenAI

client = OpenAI(api_key="")  # Replace with your actual API key

assistant = client.beta.assistants.create(
    name="Pickleball Coaching Assistant",
    instructions="You are an expert in pickleball coaching and certification. Answer questions using the uploaded PPR Study Guide.",
    model="gpt-4o",
    tools=[{"type": "file_search"}]  # Enables document-based responses
)

print(f"Assistant ID: {assistant.id}")

Assistant ID: asst_Btf2j4ThkYWJA4b75xTf5ibw


In [48]:
thread = client.beta.threads.create()
print(f"Thread ID: {thread.id}")

Thread ID: thread_6ty7U9JnAR5qPcMDPWeaxPr0


In [50]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are the key skills required to become a certified pickleball coach?"
)

print(f"Message ID: {message.id}")

Message ID: msg_6t9AQOD90S9bjvGxCNTZeV9A


In [52]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

print(f"Run ID: {run.id}")


Run ID: run_sp8XQ3RMr0VYoowswciWOQ9x


In [56]:
import time

# Wait a few seconds for completion
time.sleep(5)

# Fetch response
messages = client.beta.threads.messages.list(thread_id=thread.id)

for msg in messages.data:
    print(f"{msg.role}: {msg.content[0].text.value}\n")


assistant: It seems that there hasn't been a study guide uploaded yet. Could you please upload the PPR Study Guide? This will help me to provide you with accurate information specific to the pickleball certification process.

user: What are the key skills required to become a certified pickleball coach?



In [60]:
import time
from openai import OpenAI

client = OpenAI(api_key="")  # Replace with your API key

# Create a thread (conversation session)
thread = client.beta.threads.create()

# Upload and attach the file
file = client.files.create(
    file=open("/Users/paolarivera/Documents/Ironhack/Week 8/Day 2/lab-genai-llms-openai-assistant-main/your-code/PPR_StudyGuide_V10.pdf", "rb"),
    purpose="assistants"
)

# Send a message referencing the file
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are the key skills required to become a certified pickleball coach?",
    attachments=[{"file_id": file.id}]
)

# Run the assistant
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id="your-assistant-id"  # Replace with your actual Assistant ID
)

# Wait for processing
time.sleep(5)

# Retrieve messages from the thread
messages = client.beta.threads.messages.list(thread_id=thread.id).data

# Check if we received a response
if messages:
    message_content = messages[-1].content[0].text.value  # Get latest message
    print(message_content)
else:
    print("No response received from the assistant.")


BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'attachments[0].tools'.", 'type': 'invalid_request_error', 'param': 'attachments[0].tools', 'code': 'missing_required_parameter'}}

In [68]:
from openai import OpenAI

client = OpenAI(api_key="")  # Replace with your actual API key

# Upload the file for assistant use
file = client.files.create(
    file=open('/Users/paolarivera/Documents/Ironhack/Week 8/Day 2/lab-genai-llms-openai-assistant-main/your-code/PPR_StudyGuide_V10.pdf', "rb"),  
    purpose="assistants"
)

print(f"File uploaded successfully! File ID: {file.id}")

File uploaded successfully! File ID: file-8VD4p9LxzNkdbfiWUavrtB


In [80]:
assistant = client.beta.assistants.create(
    name="Pickleball Coaching Assistant",
    instructions=(
        "You are an expert in pickleball coaching and certification. "
        "You must use the uploaded PPR Study Guide to provide accurate answers. "
        "When responding, summarize relevant information from the document instead of general knowledge. "
        "If the question is unrelated to the document, say 'I can only answer questions related to the PPR Study Guide.'"
    ),
    model="gpt-4o",
    tools=[{"type": "file_search"}]  # ✅ Ensures the assistant can search the document
)

print(f"Assistant Created! ID: {assistant.id}")

Assistant Created! ID: asst_Xp9Lc5xY7BYy8IAaJg0SsBox


In [86]:
# Create a conversation thread
thread = client.beta.threads.create()

# ✅ FIXED: Attach file correctly (tools must be an object, not a string)
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Based on the PPR Study Guide, what are the key skills required to become a certified pickleball coach? "
            "Please provide a detailed response using the document.",
    attachments=[{"file_id": file.id, "tools": [{"type": "file_search"}]}]  # ✅ Ensures file is referenced
)

print(f"Message sent! Message ID: {message.id}")

Message sent! Message ID: msg_J9ciWrabfPyu4oDTrr8pzWJD


In [88]:
import time

# Run the assistant
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# Wait a few seconds for processing
time.sleep(5)

# Retrieve the assistant's response
messages = client.beta.threads.messages.list(thread_id=thread.id).data

if messages:
    message_content = messages[-1].content[0].text.value  # Get latest response
    print("Assistant Response:\n", message_content)
else:
    print("No response received from the assistant.")

Assistant Response:
 Based on the PPR Study Guide, what are the key skills required to become a certified pickleball coach? Please provide a detailed response using the document.


In [91]:
retrieval_logs = client.beta.threads.runs.list(thread_id=thread.id).data
print("Retrieval Logs:\n", retrieval_logs)


Retrieval Logs:
 [Run(id='run_cO7Ue0Z8zmGysFiHhkZPqQvc', assistant_id='asst_Xp9Lc5xY7BYy8IAaJg0SsBox', cancelled_at=None, completed_at=None, created_at=1739165339, expires_at=None, failed_at=1739165348, incomplete_details=None, instructions="You are an expert in pickleball coaching and certification. You must use the uploaded PPR Study Guide to provide accurate answers. When responding, summarize relevant information from the document instead of general knowledge. If the question is unrelated to the document, say 'I can only answer questions related to the PPR Study Guide.'", last_error=LastError(code='rate_limit_exceeded', message='Request too large for gpt-4o in organization org-qtRC3ZOVzMgSGpdWDFg3iWoG on tokens per min (TPM): Limit 30000, Requested 32956. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.'), max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', obje

In [94]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path, max_chars=4000):
    doc = fitz.open(pdf_path)
    text_chunks = []
    chunk = ""

    for page in doc:
        page_text = page.get_text("text") + "\n"
        if len(chunk) + len(page_text) < max_chars:
            chunk += page_text
        else:
            text_chunks.append(chunk)
            chunk = page_text

    if chunk:
        text_chunks.append(chunk)

    return text_chunks

pdf_path = '/Users/paolarivera/Documents/Ironhack/Week 8/Day 2/lab-genai-llms-openai-assistant-main/your-code/PPR_StudyGuide_V10.pdf'
ppr_text_chunks = extract_text_from_pdf(pdf_path)

print(f"Extracted {len(ppr_text_chunks)} chunks.")


Extracted 31 chunks.


In [96]:
def find_relevant_chunks(query, text_chunks):
    relevant_chunks = [chunk for chunk in text_chunks if query.lower() in chunk.lower()]
    return relevant_chunks[:3]  # Limit to 3 most relevant chunks

query = "key skills required to become a certified pickleball coach"
relevant_text = find_relevant_chunks(query, ppr_text_chunks)

print("Most Relevant Sections:\n", relevant_text)


Most Relevant Sections:
 []


In [98]:
from openai import OpenAI

client = OpenAI(api_key="your-api-key")  # Replace with your actual API key

assistant = client.beta.assistants.create(
    name="Pickleball Coaching Assistant",
    instructions=(
        "You are an expert in pickleball coaching and certification. "
        "Use the provided excerpts from the PPR Study Guide to answer questions. "
        "Summarize and provide only information found in the excerpts."
    ),
    model="gpt-4o"
)

thread = client.beta.threads.create()

# Send only the relevant chunks
message_content = f"Based on the following text from the PPR Study Guide, answer the question:\n\n{relevant_text}\n\nQuestion: What are the key skills required to become a certified pickleball coach?"

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=message_content
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

messages = client.beta.threads.messages.list(thread_id=thread.id).data

if messages:
    print("Assistant Response:\n", messages[-1].content[0].text.value)
else:
    print("No response received from the assistant.")


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-api-key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

## Exercise 3

Create an assistant that will call a weather API, given the user's answer and return the proper answer.

See the documentation of the weather API here: https://open-meteo.com/en/docs

In [100]:
import requests

def get_weather_forecast(latitude, longitude):
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m"
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Example usage:
forecast = get_weather_forecast(52.52, 13.41)
print(forecast)

{'latitude': 52.52, 'longitude': 13.419998, 'generationtime_ms': 0.032782554626464844, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 38.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-02-10T00:00', '2025-02-10T01:00', '2025-02-10T02:00', '2025-02-10T03:00', '2025-02-10T04:00', '2025-02-10T05:00', '2025-02-10T06:00', '2025-02-10T07:00', '2025-02-10T08:00', '2025-02-10T09:00', '2025-02-10T10:00', '2025-02-10T11:00', '2025-02-10T12:00', '2025-02-10T13:00', '2025-02-10T14:00', '2025-02-10T15:00', '2025-02-10T16:00', '2025-02-10T17:00', '2025-02-10T18:00', '2025-02-10T19:00', '2025-02-10T20:00', '2025-02-10T21:00', '2025-02-10T22:00', '2025-02-10T23:00', '2025-02-11T00:00', '2025-02-11T01:00', '2025-02-11T02:00', '2025-02-11T03:00', '2025-02-11T04:00', '2025-02-11T05:00', '2025-02-11T06:00', '2025-02-11T07:00', '2025-02-11T08:00', '2025-02-11T09:00', '2025-02-11T10:00', '2025-02-11T11:00', '2025-02-11T12:00

### If you want to, there is a hint here:

OpenAI Chatbots / Assistants have a way to respond in json format. 

Explore the function calling functionality